<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notebook-description" data-toc-modified-id="Notebook-description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notebook description</a></span></li><li><span><a href="#Functions-and-classes" data-toc-modified-id="Functions-and-classes-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Functions and classes</a></span><ul class="toc-item"><li><span><a href="#Load-dialogs" data-toc-modified-id="Load-dialogs-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load dialogs</a></span></li><li><span><a href="#Sequence-generator" data-toc-modified-id="Sequence-generator-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Sequence generator</a></span></li><li><span><a href="#Load-embeddings" data-toc-modified-id="Load-embeddings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Load embeddings</a></span></li><li><span><a href="#Preprocessor-class" data-toc-modified-id="Preprocessor-class-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Preprocessor class</a></span></li><li><span><a href="#Chat-bot-class" data-toc-modified-id="Chat-bot-class-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Chat bot class</a></span></li><li><span><a href="#Vanilla-Seq2Seq" data-toc-modified-id="Vanilla-Seq2Seq-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Vanilla Seq2Seq</a></span></li><li><span><a href="#Attention-Seq2Seq" data-toc-modified-id="Attention-Seq2Seq-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Attention Seq2Seq</a></span></li></ul></li><li><span><a href="#Workflow" data-toc-modified-id="Workflow-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Workflow</a></span><ul class="toc-item"><li><span><a href="#Loading-the-cornell-dialogs" data-toc-modified-id="Loading-the-cornell-dialogs-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Loading the cornell dialogs</a></span></li><li><span><a href="#Preprocessing-the-dialogs" data-toc-modified-id="Preprocessing-the-dialogs-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Preprocessing the dialogs</a></span><ul class="toc-item"><li><span><a href="#Inspect-raw-dialogs" data-toc-modified-id="Inspect-raw-dialogs-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Inspect raw dialogs</a></span></li><li><span><a href="#Clean-texts" data-toc-modified-id="Clean-texts-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Clean texts</a></span></li><li><span><a href="#Inspect-cleaned-dialogs" data-toc-modified-id="Inspect-cleaned-dialogs-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Inspect cleaned dialogs</a></span></li><li><span><a href="#Tokenize-sentences" data-toc-modified-id="Tokenize-sentences-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Tokenize sentences</a></span></li><li><span><a href="#Inspect-tokenized-dialogs" data-toc-modified-id="Inspect-tokenized-dialogs-3.2.5"><span class="toc-item-num">3.2.5&nbsp;&nbsp;</span>Inspect tokenized dialogs</a></span></li><li><span><a href="#Load-embeddings" data-toc-modified-id="Load-embeddings-3.2.6"><span class="toc-item-num">3.2.6&nbsp;&nbsp;</span>Load embeddings</a></span></li><li><span><a href="#Create-a-sequence-generator" data-toc-modified-id="Create-a-sequence-generator-3.2.7"><span class="toc-item-num">3.2.7&nbsp;&nbsp;</span>Create a sequence generator</a></span></li><li><span><a href="#Test-generator" data-toc-modified-id="Test-generator-3.2.8"><span class="toc-item-num">3.2.8&nbsp;&nbsp;</span>Test generator</a></span></li><li><span><a href="#Create-model(s)" data-toc-modified-id="Create-model(s)-3.2.9"><span class="toc-item-num">3.2.9&nbsp;&nbsp;</span>Create model(s)</a></span></li><li><span><a href="#Train-model" data-toc-modified-id="Train-model-3.2.10"><span class="toc-item-num">3.2.10&nbsp;&nbsp;</span>Train model</a></span></li><li><span><a href="#Chat-with-the-bot" data-toc-modified-id="Chat-with-the-bot-3.2.11"><span class="toc-item-num">3.2.11&nbsp;&nbsp;</span>Chat with the bot</a></span></li></ul></li></ul></li></ul></div>

# Notebook description
<p>This notebook deals with preprocessing the Cornell Movie Dialogs dataset and the IMDB Dialog dataset. We will need the data in a format suitable for a seq2seq model (meaning two lists of sentences).</p>

In [1]:
import numpy as np
import re
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow.keras.backend as K

/home/gloria/programs/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import (Input, Dense, Embedding, Lambda,
                                     Bidirectional, RepeatVector, Concatenate, Dot, Softmax, GaussianDropout)
from tensorflow.keras.layers import CuDNNLSTM as LSTM
from tensorflow.keras.layers import CuDNNGRU as GRU
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [3]:
# Use CPU
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [4]:
# Set this if you want to train with attentiopn to True, otherwise False
ATTENTION = True

# Functions and classes

## Load dialogs

In [5]:
def get_cornell(foldername, encoding='ISO-8859-1'):
    """
    Loads the cornell movie dialogs.
    ----------
    INPUT:
    foldername : string - the path to the movie dialogs (relative or absolute)
    ----------
    OUTPUT:
    questions  : list - a list containing preceeding phrases
    answers    : list - a list containing following phrases 
    """
    
    # Read in dialogs
    with open(os.path.join(foldername, 'movie_lines.txt'), 'r', encoding=encoding, errors='ignore') as f:
        lines = f.read().splitlines()

    with open(os.path.join(foldername, 'movie_conversations.txt'), 'r', encoding=encoding, errors='ignore') as f:
        ids = f.read().splitlines()
        
    # Create an id to line dictionary
    split_token = ' +++$+++ '
    id2line = {l.split(split_token)[0] : l.split(split_token)[4] for l in lines}
    
    # Create a list of conversations
    conversations_ids = []
    for conv in ids[:-1]:
        _conv = conv.split(split_token)[-1][1:-1].replace("'", '').replace(" ", "")
        conversations_ids.append(_conv.split(','))
    
    # Extract 'questions' and 'answers'
    questions = [id2line[ids[i]] for ids in conversations_ids for i in range(len(ids) - 1)]
    answers = [id2line[ids[i+1]] for ids in conversations_ids for i in range(len(ids) - 1)]
    
    return questions, answers

In [6]:
def get_imdb(foldername):
    """
    Loads the cornell movie dialogs.
    ----------
    INPUT:
    foldername : string - the path to the movie dialogs (relative or absolute)
    ----------
    OUTPUT:
    questions  : list - a list containing preceeding phrases
    answers    : list - a list containing following phrases 
    """
    
    questions = answers = None
    
    # TODO
    pass

## Sequence generator

In [7]:
class SequenceGenerator(tf.keras.utils.Sequence):
    """
    A custom sequence generator aimed at circumventing the MemoryError
    problem caused by creating large non-sparse numpy matrices. It uses keras backend.
    """
    def __init__(self, 
                 encoder_input, 
                 decoder_input, 
                 decoder_targets, 
                 batch_size=64, 
                 n_classes=None, 
                 shuffle=True,
                 attention=False,
                 latent_dim=None):
        
        
        self.batch_size = batch_size
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.decoder_targets = decoder_targets
        self.n_classes = n_classes
        if self.n_classes is None:
            self.n_classes = np.max(self.decoder_targets) + 1
        self.shuffle = shuffle
        self.attention = attention
        self.latent_dim = latent_dim
        if self.attention and self.latent_dim is None:
            raise ValueError('If you are using attention, generator also needs latent dim of decoder RNN!')
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(self.encoder_input.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """Generates one batch at call."""
        
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        """Sfuffle indexes after each epoch."""
        
        self.indexes = np.arange(self.encoder_input.shape[0])
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        """Generates data containing the batch samples."""
        
        # Simply select and one-hot encode the data
        encoder_input_batch = self.encoder_input[indexes, :]
        decoder_input_batch = self.decoder_input[indexes, :]
        decoder_targets_batch = self.decoder_targets[indexes, :]
        # If we are using attention, we also need to batch out the initial decoder states
        if self.attention:
            s = np.zeros((self.batch_size, self.latent_dim))
            inputs = [encoder_input_batch, decoder_input_batch, s, s]
        else:
            inputs = [encoder_input_batch, decoder_input_batch]
        outputs = to_categorical(decoder_targets_batch, num_classes=self.n_classes) 
        return (inputs, outputs)
    
    def test_generator(self, preprocessor, n_steps=3):
        """A function to test if generator is meaningful."""
        
        for i, batch in enumerate(self):
            
            (inps_batch, targets_batch) = batch
        
            print('*' * 10)
            print('Batch ', i+1)
            # If attention, we need to unpack 4 values, since the cell and hidden states are also inputs
            if self.attention:
                encoder_input_batch, decoder_input_batch, _, _ = inps_batch
            else:
                encoder_input_batch, decoder_input_batch = inps_batch
            print('Shape of encoder_input_batch: ', encoder_input_batch.shape)
            print('Shape of decoder_input_batch: ', decoder_input_batch.shape)
            print('Shape of targets_batch: ', targets_batch.shape)

            # Check dialogs
            print('Dialogs: ')
            for d in range(encoder_input_batch.shape[0]):
                print(preprocessor.decode_input_sequence(encoder_input_batch[d]))
                print(preprocessor.decode_output_sequence(decoder_input_batch[d]))
                print('-' * 20)  

            # Break after n steps
            if (i+1) % n_steps == 0:
                break
                self.on_epoch_end()

## Load embeddings

In [8]:
def load_embeddings(glove_dir, word_index, max_words, embedding_dim=100):
    """
    A function to load pre-trained embeddings and return them as a N x M numpy array.
    --------
    INPUT:
    glove_dir : string  - the path (relative or absolute to the glove vector text files)
    
    word_index : dict   - a word - index mapping obtained by a keras Tokenizer or manually
    
    max_words : int     - the maximum number of words in the vocabulary used for the application
    
    embedding_dim : int - the dimensionality of the embedding space
    ---------
    OUTPUT:
    embedding_matrix : (max_words, embedding_dim) ndarray - the matrix of embeddings which can be
                                                            directly loaded into an Embedding layer
    """
    
    
    # Create a word - vec dictionary from the pre-trained glvoe vectors
    embeddings_index = {}
    with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.array(values[1:], dtype='float32')
            embeddings_index[word] = coefs
            
    # Now we will build an embedding matrix that we can load into an Embedding layer.
    # It iwll be of shape (max_words, embedding_dim), where each entry i will contain
    # the embedding for the word of index i in the reference built during tokenization.
    # Index 0 is a placeholder!
    
    embedding_matrix = np.zeros((max_words, embedding_dim))
    for word, i in word_index.items():
        if i < max_words:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector
                
    return embedding_matrix

## Preprocessor class

In [9]:
class Seq2SeqPreprocessor:
    """
    A class to bundle utility functions useful for preprocessing nlp data for a seq2seq model.
    """
    
    def __init__(self, max_seq_len=50, vocab_size=20000):
        """
        Initializes preprocessor instance.
        ----------
        INPUT:
        max_seq_len : int - defines the longest sentence allowed. If a Q or an A in a Q&A pair
                            contains more than max_seq_len, it will be later discarded.
                            
        vocab_size  : int - the maximum number of words in the vocabulary. Used by keras' Tokenizer
        """
        
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.input_tokenizer = Tokenizer(num_words=vocab_size, filters='')
        self.output_tokenizer = Tokenizer(num_words=vocab_size, filters='')
        self.idx2word_input = {}
        self.idx2word_output = {}
        
    def clean_text(self, text):
        """
        A custom preprocessing of the input texts. It lowers the text, replaces all short 
        verb forms with their full counterparts (I'm => I am), and removes all non-alphanumeric 
        characters as well as multiple spaces with single spaces.
        ----------
        INPUT:
        text : string - a line of text to be cleaned
        ----------
        OUTPUT
        text : string - the cleaned line of text
        """
        
        # Put all lowercase
        text = text.lower()
        
        # Pronouns
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"it's", "it is", text)
        
        # Questions
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"why's", "why is", text)
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"here's", "here is", text)
        
        # Negations
        text = re.sub(r"hasn't", "has not", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"wouldn't", "would not", text)
        text = re.sub(r"shouldn't", "should not", text)
        text = re.sub(r"mustn't", "must not", text)
        text = re.sub(r"haven't", "have not", text)
        text = re.sub(r"hadn't", "had not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"didn't", "did not", text)
        text = re.sub(r"doesn't", "does not", text)
        text = re.sub(r"don't", "do not", text)
        text = re.sub(r"aren't", "are not", text)
        
        # Modals
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        
        # Keep only alphanumeric and space
        text = re.sub(r"([^\s\w]|_)+", "", text)
        # Keep only single spaces
        text = re.sub('\s+', ' ', text).strip()
        return text
    
    def _add_sos(self, text):
        return '<sos> ' + text 

    def _add_eos(self, text):
        return text + ' <eos>'  
    
    def clean_texts(self, inputs, targets):
        """
        Applies clean_text() to each input/targets pair. Adds EOS token to inputs and SOS to targets.
        ----------
        INPUT:
        inputs  - list        : a list of input sentences (strings)
        targets - list        : a list of target sentences (strings)
        ----------
        OUTPUT:
        inputs_cleaned        : list - a list of the cleaned input sentences
        targets_cleaned       : list - a list of the cleaned target sentences used as decoder output
        targets_input_cleaned : list - a list of the cleaned sentences used as decoder input (teacher forcing)
        """
        
        assert len(inputs) == len(targets), 'Input and target length must match!'
        
        # Clean inputs and targets
        inputs_cleaned = [self.clean_text(inp) for inp in inputs]
        targets_cleaned = [self.clean_text(t) for t in targets]
    
        # Remove sequences larger than max sequence length
        # First figure out indices of sentences to be removed
        idx_to_remove = []
        for i in range(len(inputs_cleaned)):
            inp_len = len(inputs_cleaned[i].split())
            target_len = len(targets_cleaned[i].split()) - 1 # we will later add eos and sos
            if inp_len > self.max_seq_len or target_len > self.max_seq_len - 1:
                idx_to_remove.append(i)
        
        # Then remove (not in-place)
        inputs_cleaned = np.delete(inputs_cleaned, idx_to_remove)
        targets_cleaned = np.delete(targets_cleaned, idx_to_remove)
        
        # Add EOS and SOS tokens, creating the input_targets list (no need for sos in decoder inputs)
        targets_cleaned = [self._add_eos(t) for t in targets_cleaned]
        targets_input_cleaned = [self._add_sos(t.replace('<eos>', '')) for t in targets_cleaned]
        
        return inputs_cleaned, targets_cleaned, targets_input_cleaned
    
    def tokenize_texts(self, inputs, targets, input_targets):
        """
        Tokenizes the three lists of cleaned texsts using keras' Tokenizer, and padds with zeros
        ----------
        INPUT:
        inputs           : list - a list of cleaned input texts (for encoder inpout)
        targets          : list - a list of cleaned target texts with EOS (decoder output)
        input_targets    : list - a list of cleaned target input texts wit SOS (decoder input)
        ----------
        OUTPUT:
        input_sequences  : np.ndarray - a N x max_seq_len numpy array with tokenized 
                                            and padded input sentences
        target_sequences : np.ndarray - a N x max_seq_len numpy array with tokenized 
                                            and padded target sentences
        target_input_sequences : np.ndarray - a N x max_seq_len numpy array with tokenized 
                                            and padded target input sentences
        """
        
        # Fit tokenizers
        self._fit_tokenizers(inputs, targets, input_targets)
        
        # Transform to sequences + padding
        input_sequences = self.tokenize_input(inputs)
        target_sequences = self.tokenize_output(targets)
        target_input_sequences = self.tokenize_output(input_targets)
        
        return input_sequences, target_sequences, target_input_sequences
        
    def _fit_tokenizers(self, inputs, targets, input_targets):
        """Utility function to separate fitting from transforming texts to sequences."""
        
        # Fit tokenizers
        self.input_tokenizer.fit_on_texts(inputs)
        self.output_tokenizer.fit_on_texts(targets + input_targets)
        
        # Obtain index 2 word dictionaries
        self.idx2word_input = {v : k for k, v in self.input_tokenizer.word_index.items()}
        self.idx2word_output = {v : k for k, v in self.output_tokenizer.word_index.items()}
        self.word2idx_input = self.input_tokenizer.word_index
        self.word2idx_output = self.output_tokenizer.word_index
        
    def tokenize_input(self, inputs):
        return pad_sequences(self.input_tokenizer.texts_to_sequences(inputs), maxlen=self.max_seq_len)
    
    def tokenize_output(self, outputs):
        return pad_sequences(self.output_tokenizer.texts_to_sequences(outputs), maxlen=self.max_seq_len, padding='post')
        
    def decode_input_sequence(self, sequence):
        """Decodes a tokenized and padded input sequence."""
        
        return ' '.join([self.idx2word_input[idx] for idx in sequence if idx != 0])
    
    def decode_output_sequence(self, sequence):
        """Decodes a tokenized and padded output sequence."""
        
        return ' '.join([self.idx2word_output[idx] for idx in sequence if idx != 0])
    
    def test_random_dialogs_raw(self, inputs, targets, num_exchanges=5):
        """A function to test by inspection if the raw dialogs are properly aligned."""

        indices = np.random.randint(0, len(inputs), num_exchanges)

        for i, idx in enumerate(indices):
            print('Random dialog {}:'.format(idx))
            print(inputs[idx])
            print(targets[idx])
            print('-' * 10)
            
    def test_random_dialogs_preprocessed(self, inputs, targets, num_exchanges=5):
        """A function to test by inspection if the preprocessed dialogs are properly aligned."""
        
        indices = np.random.randint(0, inputs.shape[0], num_exchanges)
        
        for i, idx in enumerate(indices):
            print('Random dialog {}:'.format(idx))
            print(self.decode_input_sequence(inputs[idx]))
            print(self.decode_output_sequence(targets[idx]))
            print('-' * 10)  

## Chat bot class

In [10]:
class ChatBot:
    """The ready made chat bot class utilized after training of the seq2seq model."""
    def __init__(self, preprocessor, training_model, encoder, decoder, attention, latent_dim=128):
        
        self.preprocessor = preprocessor
        self.training_model = training_model
        self.encoder = encoder
        self.decoder = decoder
        self.attention = attention
        self.latent_dim = latent_dim
        
    def train_model(self, train_gen, val_gen, epochs, save_name, verbose=True):
        """Intereface to optimize model."""
        
        # Use checkpointing
        filepath="./checkpoints/weights-{}-best.hdf5".format(save_name)
        checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=True, save_best_only=True, mode='max')
        tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
        
        # Fit
        h = self.training_model.fit_generator(train_gen, 
                                              validation_data=val_gen, 
                                              epochs=epochs, 
                                              verbose=verbose,
                                              callbacks=[checkpoint,tb])
        
    def test_model(self, test_sentences, target_sentences, n_sentences=3):
        """Judge visually the responses of the model on test sentences (test set or custom)."""
        
        # Choose random indices.
        indices = np.random.randint(0, test_sentences.shape[0], size=n_sentences)
        for i, idx in enumerate(indices):
            decoded_input_sentence = self.preprocessor.decode_input_sequence(test_sentences[idx, :])
            decoded_target_sentence = self.preprocessor.decode_output_sequence(target_sentences[idx, :])
            response = self.answer(decoded_input_sentence)
            
            print('Test ', i+1)
            print('Input sentence: ')
            print(decoded_input_sentence)
            print('Chat bot output: ')
            print(response)
            print('Expected target sentence: ')
            print(decoded_target_sentence)
            print('-' * 20)
    
    def chat(self):
        """Chat interface."""
        
        print("Hi, let's talk!")
        print("To end the conversation, enter an empty line. :)")
        while True:
            txt = input('You: ')
            if not txt:
                print('Bye!')
                break
            if self.attention:
                response = self.answer_attention(txt)
            else:
                response = self.answer(txt)
            print(response)
            
    def answer_attention(self, question):
        """Answer a question with attention."""
        
        text = self.preprocessor.clean_text(question)
        encoder_output = self.encoder.predict(self.preprocessor.tokenize_input([text]))
        target_seq = np.zeros((1, 1))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0] = self.preprocessor.word2idx_output['<sos>']
        eos_idx = self.preprocessor.word2idx_output['<eos>']
        # Initial states
        s, c = np.zeros((1, self.latent_dim)), np.zeros((1, self.latent_dim))
        
        # Decoder loop
        output_sentence = []
        for _ in range(self.preprocessor.max_seq_len):
            o, s, c = self.decoder.predict([target_seq, encoder_output, s, c])
            # Index of next word
            idx = np.argmax(o.flatten())
            
            #If EOS, end sentence
            if idx == eos_idx:
                break
            if idx > 0:
                word = self.preprocessor.idx2word_output[idx]
                output_sentence.append(word)
            # Update the decoder input which is just the word previously generated
            target_seq[0, 0] = idx
        # Try to guess if question or no (we coult have learnt that, just use for prettines)
        if output_sentence[0] in ['what', 'where', 'how', 'who', 'why', 'when', 'which', 'are']:
            end_char = '?'
        else:
            end_char = '.'
        # Return pretty sentence
        return ' '.join(output_sentence).capitalize() + end_char
        
    def answer(self, question):
        """Answer a question."""
        
        # Clean (just in case)
        text = self.preprocessor.clean_text(question)
        
        # Encode the input as state vectors.
        states = self.encoder.predict(self.preprocessor.tokenize_input([text]))
        
        # Generate empty target sequence of length 1 (language generation model)
        target_seq = np.zeros((1, 1))

        # Populate the first character of target sequence with the start character.
        target_seq[0, 0] = self.preprocessor.word2idx_output['<sos>']
        eos_idx = self.preprocessor.word2idx_output['<eos>']
        
        # Output sequence
        output_sentence = []
        for _ in range(self.preprocessor.max_seq_len):
            output_tokens, h, c = self.decoder.predict([target_seq] + states)
            # Get next word (most probable)
            idx = np.argmax(output_tokens[0, 0, :])
            #If EOS, end sentence
            if idx == eos_idx:
                break
            if idx > 0:
                word = self.preprocessor.idx2word_output[idx]
                output_sentence.append(word)
            # Update the decoder input which is just the word previously generated
            target_seq[0, 0] = idx
            # Update states
            states = [h, c]
        # Try to guess if question or no (we coult have learnt that, just use for prettines)
        if output_sentence[0] in ['what', 'where', 'how', 'who', 'why', 'when', 'which', 'are']:
            end_char = '?'
        else:
            end_char = '.'
        # Return pretty sentence
        return ' '.join(output_sentence).capitalize() + end_char
    
    def _temperature(self, output_tokens):
        """Reweight softmax distribution."""
        
        pass

## Vanilla Seq2Seq

In [11]:
def create_seq2seq(preprocessor, lstm_dim=128, embedding_dim=100, 
                  dropout_rate=0.5, encoder_embeddings=None, decoder_embeddings=None):
    """Creates the seq2seq model for training and prediction."""
    
    ### TRAINING

    # Encoder input
    encoder_input = Input(shape=(preprocessor.max_seq_len, ))

    # Encoder embedding
    encoder_embedding = Embedding(
                    input_dim=preprocessor.vocab_size, 
                    output_dim=embedding_dim,
                    weights=[encoder_embeddings],
                    input_length=preprocessor.max_seq_len,
                    trainable=False
    )
    encoder_X = encoder_embedding(encoder_input)

    # Encoder LSTM
    encoder_drop = GaussianDropout(dropout_rate)(encoder_X)
    encoder_lstm = LSTM(lstm_dim, return_state=True)
    encoder_outputs, h, c = encoder_lstm(encoder_drop)

    # "Thought vector"
    encoder_states = [h, c]

    # Decoder input
    decoder_input = Input(shape=(preprocessor.max_seq_len, ))

    # Decoder embedding
    decoder_embedding = Embedding(
                input_dim=preprocessor.vocab_size, 
                output_dim=embedding_dim,
                weights=[decoder_embeddings],
                input_length=preprocessor.max_seq_len,
                trainable=False
    )      
    decoder_X = decoder_embedding(decoder_input)

    # Decoder LSTM
    decoder_drop = GaussianDropout(dropout_rate)(decoder_X)
    decoder_lstm = LSTM(lstm_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_drop, initial_state=encoder_states)

    # Softmax classifier
    decoder_dense = Dense(preprocessor.vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model definition and compilation
    training_model = Model([encoder_input, decoder_input], decoder_outputs)
    
    ### PREDICTION MODEL
    
    # The encoder will output the initial decoder hidden state
    encoder_model = Model(encoder_input, encoder_states)

    decoder_state_input_h = Input(shape=(lstm_dim,))
    decoder_state_input_c = Input(shape=(lstm_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_input_single = Input(shape=(1,))
    decoder_input_single_x = decoder_embedding(decoder_input_single)

    # this time, we want to keep the states too, to be output
    # by our sampling model
    decoder_outputs, h, c = decoder_lstm(
      decoder_input_single_x,
      initial_state=decoder_states_inputs
    )

    decoder_states = [h, c]

    decoder_outputs = decoder_dense(decoder_outputs)

    # The sampling model
    decoder_model = Model(
      [decoder_input_single] + decoder_states_inputs, 
      [decoder_outputs] + decoder_states
    )

    # Compile the training model
    training_model.compile(
      optimizer='rmsprop',
      loss='categorical_crossentropy',
      metrics=['accuracy']
    )
    
    return training_model, encoder_model, decoder_model

## Attention Seq2Seq

In [12]:
def create_attention_seq2seq(preprocessor, lstm_dim=128, embedding_dim=100, 
                  dropout_rate=0.5, encoder_embeddings=None, decoder_embeddings=None):
    """Creates the seq2seq model with attention for training and prediction."""
    
    
    ### TRAINING ###
    # -------------------------------- #
    
    ## ENCODER ##
    # Encoder input
    encoder_input = Input(shape=(preprocessor.max_seq_len, ))

    # Encoder embedding
    encoder_embedding = Embedding(
                    input_dim=preprocessor.vocab_size, 
                    output_dim=embedding_dim,
                    weights=[encoder_embeddings],
                    input_length=preprocessor.max_seq_len,
                    trainable=False
    )
    encoder_X = encoder_embedding(encoder_input)
    
    # Encoder is a bidirection lstm
    encoder_drop = GaussianDropout(dropout_rate)(encoder_X)
    encoder_lstm = Bidirectional(LSTM(lstm_dim, return_sequences=True))
    encoder_outputs  = encoder_lstm(encoder_drop)
    
    ## DECODER ##
    # Decoder input
    decoder_input = Input(shape=(preprocessor.max_seq_len, ))

    # Decoder embedding
    decoder_embedding = Embedding(
                input_dim=preprocessor.vocab_size, 
                output_dim=embedding_dim,
                weights=[decoder_embeddings],
                input_length=preprocessor.max_seq_len,
                trainable=False
    )      
    decoder_X = decoder_embedding(decoder_input)
    
    # Attention mechanism
    attn_repeat_layer = RepeatVector(preprocessor.max_seq_len)
    attn_concat_layer = Concatenate(axis=-1)
    attn_dense1 = Dense(12, activation='tanh')
    attn_dense2 = Dense(1)
    attn_softmax = Softmax(axis=1)
    attn_dot = Dot(axes=1)
    
    # Decoder LSTM
    decoder_lstm = LSTM(lstm_dim, return_state=True)
    decoder_dense = Dense(preprocessor.vocab_size, activation='softmax')
    initial_s = Input(shape=(lstm_dim, ), name='s0') # hidden state
    initial_c = Input(shape=(lstm_dim, ), name='c0') # cell state
    context_last_word_concat = Concatenate(axis=2) # use for teacher forcing
    
    # Now we obtain the output in Ty steps
    # In each step, we consider all encoder outputs and current s, and c
    s = initial_s
    c = initial_c
    
    outputs = []
    # Loop for each output step
    for t in range(preprocessor.max_seq_len):
        
        # Get context for step t
        st_1 = attn_repeat_layer(s)
        x_att = attn_concat_layer([encoder_outputs, st_1])
        alphas = attn_dense1(x_att)
        alphas = attn_dense2(alphas)
        alphas = attn_softmax(alphas)
        context = attn_dot([alphas, encoder_outputs])
        
        # Select previous word for teacher forcing
        selector = Lambda(lambda x: x[:, t:t+1])
        target_x_t = selector(decoder_X)
        
        # Combine context with previous word and get decoder output at step t
        decoder_X_t = context_last_word_concat([context, target_x_t])
        o, s, c = decoder_lstm(decoder_X_t, initial_state=[s, c])
        
        # Get next word prediction and add to outputs list
        decoder_output = decoder_dense(o)
        outputs.append(decoder_output)
        
    # Combine all the outputs from the list 
    # The list now contains Ty NxD matrices
    # where N is the batch_size and D is the vocab_size
    # Thus, we want a NxTyxD tensor => permute the stacked tensor which is TyxNxD
    output_combiner = Lambda(lambda x: K.permute_dimensions(K.stack(x), pattern=(1, 0, 2)), name='Combiner')
    outputs_stacked = output_combiner(outputs)
    
    # Create and compile the training model
    training_model = Model(
            inputs=[encoder_input, decoder_input, initial_s, initial_c],
            outputs=outputs_stacked
    )
    
    training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    
    ### PREDICTION ###

    # Encoder
    encoder_model = Model(encoder_input, encoder_outputs)
    
    # Decoder (with Ty = 1)
    encoder_output_as_input = Input(shape=(preprocessor.max_seq_len, lstm_dim * 2, ))
    decoder_input_single = Input(shape=(1,))
    decoder_input_single_x = decoder_embedding(decoder_input_single)
    
    # Attention
    st_1 = attn_repeat_layer(initial_s)
    x_att = attn_concat_layer([encoder_output_as_input, st_1])
    alphas = attn_dense1(x_att)
    alphas = attn_dense2(alphas)
    alphas = attn_softmax(alphas)
    context = attn_dot([alphas, encoder_output_as_input])
    
    # Decoder lstm
    decoder_lstm_input = context_last_word_concat([context, decoder_input_single_x])
    o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
    decoder_outputs = decoder_dense(o)
    
    decoder_model = Model(
        inputs=[decoder_input_single, encoder_output_as_input, initial_s, initial_c], 
        outputs=[decoder_outputs, s, c]
    )
    
    return training_model, encoder_model, decoder_model    

# Workflow

## Loading the cornell dialogs

In [13]:
questions, answers = get_cornell('dialogs/cornell_movie-dialogs_corpus/', encoding='utf-8')

## Preprocessing the dialogs

In [14]:
preprocessor = Seq2SeqPreprocessor(vocab_size=20000)

### Inspect raw dialogs

In [15]:
preprocessor.test_random_dialogs_raw(questions, answers)

Random dialog 202902:
Nazis talk about it all the time.
Do they?
----------
Random dialog 125104:
What?
The doctor said you were having sex.
----------
Random dialog 173702:
Where's Elias come from?
'Lias come naturally.
----------
Random dialog 22662:
I promise.
Kate, Scott?
----------
Random dialog 102652:
Are you sure this guy is cool?
You'll see for yourself.
----------


### Clean texts

In [16]:
inputs_cleaned, targets_cleaned, targets_input_cleaned = preprocessor.clean_texts(questions, answers)

### Inspect cleaned dialogs

In [17]:
preprocessor.test_random_dialogs_raw(inputs_cleaned, targets_cleaned)

Random dialog 6214:
laws do not touch men like thaddeus rains only justice does
whose justice yours or gods when will you stop <eos>
----------
Random dialog 153887:
it it does not matter mother he is dead now dead
yes dead two nights ago the night of my dream <eos>
----------
Random dialog 207431:
all of it about him an you an uncle pete about robbin the rock island pacific an about them missouri banks
we done that stuff <eos>
----------
Random dialog 99099:
no i think i will stay right here in hell
kind of a bleak point of view there isnt it marty <eos>
----------
Random dialog 158133:
hows it going everything okay
just like old times <eos>
----------


### Tokenize sentences

In [18]:
input_sequences, target_sequences, target_input_sequences = preprocessor.tokenize_texts(inputs_cleaned, 
                                                                                        targets_cleaned, 
                                                                                        targets_input_cleaned)

In [19]:
print(input_sequences.shape)
print(target_sequences.shape)
print(target_input_sequences.shape)

(214903, 50)
(214903, 50)
(214903, 50)


In [20]:
# Use some splits as validation
(input_sequences_train, 
input_sequences_test, 
target_sequences_train, 
target_sequences_test, 
target_input_sequences_train, 
target_input_sequences_test) = train_test_split(input_sequences, 
                                                target_sequences, 
                                                target_input_sequences, 
                                                test_size=0.025, 
                                                random_state=42)

#(input_sequences_test,
#input_sequences_dev,
#target_sequences_test, 
#target_sequences_dev, 
#target_input_sequences_test, 
#target_input_sequences_dev) = train_test_split(input_sequences_test, 
#                                                target_sequences_test, 
#                                                target_input_sequences_test, 
#                                                test_size=0.5, 
#                                                random_state=42)

In [21]:
print('Shape of input_sequences_train: ', input_sequences_train.shape)
print('Shape of target_sequences_train: ', target_sequences_train.shape)
print('Shape of target_input_sequences_train: ', target_input_sequences_train.shape)
print('-' * 20)
print('Shape of input_sequences_test: ', input_sequences_test.shape)
print('Shape of target_sequences_test: ', target_sequences_test.shape)
print('Shape of target_input_sequences_test: ', target_input_sequences_test.shape)
print('-' * 20)
#print('Shape of target_sequences_dev: ', target_sequences_dev.shape)
#print('Shape of input_sequences_dev: ', input_sequences_dev.shape)
#print('Shape of target_input_sequences_dev: ', target_input_sequences_dev.shape)

Shape of input_sequences_train:  (209530, 50)
Shape of target_sequences_train:  (209530, 50)
Shape of target_input_sequences_train:  (209530, 50)
--------------------
Shape of input_sequences_test:  (5373, 50)
Shape of target_sequences_test:  (5373, 50)
Shape of target_input_sequences_test:  (5373, 50)
--------------------


### Inspect tokenized dialogs

In [22]:
preprocessor.test_random_dialogs_preprocessed(input_sequences, target_sequences)

Random dialog 88300:
come on
no <eos>
----------
Random dialog 61609:
i can just hear you now get out of here you perfect wonderful woman you are too good for me
that is the hard part <eos>
----------
Random dialog 187080:
i am eighteen it is my decision i think i would make a pretty good pilot i have good great reflexes you need that for maneuvering you know
only 1 in 14 male applicants make it through stellar navigation so what do you think your chances are <eos>
----------
Random dialog 127828:
uhhuh a what
shep proudfoot that is a name <eos>
----------
Random dialog 99426:
yes and no did you recognize him
no i only saw his back he went down another stairwell at the end of the hall <eos>
----------


### Load embeddings

In [23]:
# Load separately, since we might have different words in input, output
encoder_embeddings = load_embeddings('embeddings', 
                                     preprocessor.word2idx_input, 
                                     preprocessor.vocab_size)


decoder_embeddings = load_embeddings('embeddings', 
                                     preprocessor.word2idx_output, 
                                     preprocessor.vocab_size)

print('Shape of encoder embeddings: ', encoder_embeddings.shape)
print('Shape of decoder embeddings: ', decoder_embeddings.shape)

Shape of encoder embeddings:  (20000, 100)
Shape of decoder embeddings:  (20000, 100)


### Create a sequence generator

In [24]:
# Create generators for batch sampling

# Train generator
seq_gen_train = SequenceGenerator(input_sequences_train, 
                            target_input_sequences_train, 
                            target_sequences_train, 
                            batch_size=64, 
                            n_classes=preprocessor.vocab_size,
                            attention=ATTENTION,
                            latent_dim=128)

# Validation generator
seq_gen_val = SequenceGenerator(input_sequences_test, 
                            target_input_sequences_test, 
                            target_sequences_test, 
                            batch_size=64, 
                            n_classes=preprocessor.vocab_size,
                            attention=ATTENTION,
                            latent_dim=128)

# Development generator (for testing model with small data samples)
#seq_gen_dev = SequenceGenerator(input_sequences_dev, 
#                            target_input_sequences_dev, 
#                            target_sequences_dev, 
#                            batch_size=64, 
#                            n_classes=preprocessor.vocab_size,
#                            attention=ATTENTION,
#                            latent_dim=128)

### Test generator

In [25]:
# seq_gen_dev.test_generator(preprocessor)

### Create model(s)

In [26]:
K.clear_session()


if ATTENTION:
    # Attention seq2seq
    model, encoder_model, decoder_model = create_attention_seq2seq(preprocessor, 
                                                    encoder_embeddings=encoder_embeddings, 
                                                    decoder_embeddings=decoder_embeddings)
else:
    # Vanilla seq2seq
    model, encoder_model, decoder_model = create_seq2seq(preprocessor, 
                                                        encoder_embeddings=encoder_embeddings, 
                                                        decoder_embeddings=decoder_embeddings)



In [27]:
# decoder_model.summary()

### Train model 

In [28]:
chatbot = ChatBot(preprocessor, model, encoder_model, decoder_model, attention=ATTENTION)
chatbot.train_model(seq_gen_train, seq_gen_val, save_name='seq2seq_attention', epochs=10)

Epoch 1/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.1871 - acc: 0.8187
Epoch 00001: val_acc improved from -inf to 0.82736, saving model to ./checkpoints/weights-seq2seq_attention-best.hdf5


3273/3273 [==============================] - 2541s 776ms/step - loss: 1.1871 - acc: 0.8187 - val_loss: 1.0782 - val_acc: 0.8274
Epoch 2/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.1502 - acc: 0.8197
Epoch 00002: val_acc improved from 0.82736 to 0.82897, saving model to ./checkpoints/weights-seq2seq_attention-best.hdf5


3273/3273 [==============================] - 2497s 763ms/step - loss: 1.1502 - acc: 0.8197 - val_loss: 1.0633 - val_acc: 0.8290
Epoch 3/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0686 - acc: 0.8274
Epoch 00003: val_acc improved from 0.82897 to 0.83069, saving model to ./checkpoints/weights-seq2seq_attention-best.hdf5


3273/3273 [==============================] - 2502s 764ms/step - loss: 1.0686 - acc: 0.8274 - val_loss: 1.0489 - val_acc: 0.8307
Epoch 4/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0644 - acc: 0.8284
Epoch 00004: val_acc improved from 0.83069 to 0.83173, saving model to ./checkpoints/weights-seq2seq_attention-best.hdf5


3273/3273 [==============================] - 2501s 764ms/step - loss: 1.0645 - acc: 0.8284 - val_loss: 1.0471 - val_acc: 0.8317
Epoch 5/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0612 - acc: 0.8291
Epoch 00005: val_acc improved from 0.83173 to 0.83182, saving model to ./checkpoints/weights-seq2seq_attention-best.hdf5


3273/3273 [==============================] - 2503s 765ms/step - loss: 1.0612 - acc: 0.8291 - val_loss: 1.0463 - val_acc: 0.8318
Epoch 6/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0644 - acc: 0.8294
Epoch 00006: val_acc did not improve from 0.83182
3273/3273 [==============================] - 2504s 765ms/step - loss: 1.0644 - acc: 0.8294 - val_loss: 1.0563 - val_acc: 0.8315
Epoch 7/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0644 - acc: 0.8297
Epoch 00007: val_acc did not improve from 0.83182
3273/3273 [==============================] - 2503s 765ms/step - loss: 1.0645 - acc: 0.8297 - val_loss: 1.0555 - val_acc: 0.8318
Epoch 8/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0673 - acc: 0.8297
Epoch 00008: val_acc did not improve from 0.83182
3273/3273 [==============================] - 2508s 766ms/step - loss: 1.0674 - acc: 0.8297 - val_loss: 1.0602 - val_acc: 0.8316
Epoch 9/10
3272/3273 [============================>.] - ETA

3273/3273 [==============================] - 2512s 768ms/step - loss: 1.0670 - acc: 0.8299 - val_loss: 1.0586 - val_acc: 0.8323
Epoch 10/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0680 - acc: 0.8298
Epoch 00010: val_acc did not improve from 0.83226
3273/3273 [==============================] - 2516s 769ms/step - loss: 1.0681 - acc: 0.8298 - val_loss: 1.0630 - val_acc: 0.8320


In [29]:
model.evaluate_generator(seq_gen_val)

[1.0610862740551132, 0.8323719853378204]

In [30]:
model.save_weights('seq2seq_attention_weights.h5')

In [31]:
model2, encoder_model2, decoder_model2 = create_attention_seq2seq(preprocessor, 
                                                        encoder_embeddings=encoder_embeddings, 
                                                        decoder_embeddings=decoder_embeddings)

In [32]:
model2.load_weights('seq2seq_attention_weights.h5')

In [33]:
model2.evaluate_generator(seq_gen_val)

[1.0607992705092373, 0.8324096360838557]

In [34]:
encoder_model.save_weights('seq2seq_attention_encoder_weights.h5')

In [35]:
decoder_model.save_weights('seq2seq_attention_decoder_weights.h5')

### Chat with the bot

### After 10 epochs

In [37]:
chatbot.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi
Hi.
You: How are you?
I am not going to be a little bit.
You: That's okay.
I am sorry i am not going to be a little bit.
You: Where are you?
I am not going to get it.
You: Get what?
I am not going to be a little bit.
You: 
Bye!


### Continue training for another 10 epochs

In [40]:
chatbot.train_model(seq_gen_train, seq_gen_val, save_name='seq2seq_attention_2', epochs=10)

Epoch 1/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0659 - acc: 0.8300
Epoch 00001: val_acc improved from -inf to 0.83195, saving model to ./checkpoints/weights-seq2seq_attention_2-best.hdf5


3273/3273 [==============================] - 2547s 778ms/step - loss: 1.0659 - acc: 0.8300 - val_loss: 1.0597 - val_acc: 0.8320
Epoch 2/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0661 - acc: 0.8301
Epoch 00002: val_acc improved from 0.83195 to 0.83232, saving model to ./checkpoints/weights-seq2seq_attention_2-best.hdf5


3273/3273 [==============================] - 2524s 771ms/step - loss: 1.0661 - acc: 0.8301 - val_loss: 1.0601 - val_acc: 0.8323
Epoch 3/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0669 - acc: 0.8302
Epoch 00003: val_acc improved from 0.83232 to 0.83252, saving model to ./checkpoints/weights-seq2seq_attention_2-best.hdf5


3273/3273 [==============================] - 2533s 774ms/step - loss: 1.0669 - acc: 0.8302 - val_loss: 1.0563 - val_acc: 0.8325
Epoch 4/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0638 - acc: 0.8301
Epoch 00004: val_acc did not improve from 0.83252
3273/3273 [==============================] - 2544s 777ms/step - loss: 1.0638 - acc: 0.8301 - val_loss: 1.0676 - val_acc: 0.8317
Epoch 5/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0639 - acc: 0.8301
Epoch 00005: val_acc did not improve from 0.83252
3273/3273 [==============================] - 2525s 771ms/step - loss: 1.0639 - acc: 0.8301 - val_loss: 1.0562 - val_acc: 0.8324
Epoch 6/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0646 - acc: 0.8301
Epoch 00006: val_acc did not improve from 0.83252
3273/3273 [==============================] - 2517s 769ms/step - loss: 1.0645 - acc: 0.8301 - val_loss: 1.0618 - val_acc: 0.8319
Epoch 7/10
3272/3273 [============================>.] - ETA

In [41]:
model.save_weights('seq2seq_attention_weights_epoch_20.h5')
encoder_model.save_weights('seq2seq_attention_encoder_weights_epoch_20.h5')
decoder_model.save_weights('seq2seq_attention_decoder_weights_epoch_20.h5')

### After 20 epochs

In [42]:
chatbot.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi
Hi.
You: How are you?
Fine.
You: Great!
I am sorry.
You: Why?
Because you are not.
You: Where are you?
I am going to get a little bit.
You: What's your name?
I am sorry.
You: Why?
Because you are not.
You: Nice to meet you.
I am sorry.
You: You don't need to be sorry.
I am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am not sure i am.
You: 
Bye!


### Compare against the model trained for 10 epochs

In [43]:
model_10, encoder_model_10, decoder_model_10 = create_attention_seq2seq(preprocessor, 
                                                        encoder_embeddings=encoder_embeddings, 
                                                        decoder_embeddings=decoder_embeddings)

In [44]:
model_10.load_weights('seq2seq_attention_weights.h5')
encoder_model_10.load_weights('seq2seq_attention_encoder_weights.h5')
decoder_model_10.load_weights('seq2seq_attention_decoder_weights.h5')

In [45]:
chatbot_10 = ChatBot(preprocessor, model_10, encoder_model_10, decoder_model_10, attention=ATTENTION)

In [46]:
chatbot_10.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi
Hi.
You: How are you?
I am not going to be a little bit.
You: That's okay.
I am sorry i am not going to be a little bit.
You: Where are you?
I am not going to get it.
You: Get what?
I am not going to be a little bit.
You: What's your name?
I am not going to be a little bit.
You: Nice to meet you.
I am not going to be a little bit.
You: You don't need to be sorry.
I am not going to be a little bit.
You: Hi
Hi.
You: Bye
Hi.
You: 
Bye!


#### Compared to training for 10 epochs, the chatbot improved in terms of response variation, despite the fact that training loss did not seem to improve significantly in the last 10 epochs.

### Continue training for another 10 epochs

In [49]:
chatbot.train_model(seq_gen_train, seq_gen_val, save_name='seq2seq_attention_3', epochs=10)

Epoch 1/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0695 - acc: 0.8298
Epoch 00001: val_acc improved from -inf to 0.83188, saving model to ./checkpoints/weights-seq2seq_attention_3-best.hdf5


3273/3273 [==============================] - 2534s 774ms/step - loss: 1.0695 - acc: 0.8298 - val_loss: 1.0642 - val_acc: 0.8319
Epoch 2/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0708 - acc: 0.8298
Epoch 00002: val_acc improved from 0.83188 to 0.83228, saving model to ./checkpoints/weights-seq2seq_attention_3-best.hdf5


3273/3273 [==============================] - 2520s 770ms/step - loss: 1.0708 - acc: 0.8298 - val_loss: 1.0676 - val_acc: 0.8323
Epoch 3/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0728 - acc: 0.8298
Epoch 00003: val_acc did not improve from 0.83228
3273/3273 [==============================] - 2544s 777ms/step - loss: 1.0727 - acc: 0.8299 - val_loss: 1.0685 - val_acc: 0.8319
Epoch 4/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0746 - acc: 0.8299
Epoch 00004: val_acc did not improve from 0.83228
3273/3273 [==============================] - 2539s 776ms/step - loss: 1.0746 - acc: 0.8299 - val_loss: 1.0720 - val_acc: 0.8318
Epoch 5/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0767 - acc: 0.8298
Epoch 00005: val_acc improved from 0.83228 to 0.83250, saving model to ./checkpoints/weights-seq2seq_attention_3-best.hdf5


3273/3273 [==============================] - 2550s 779ms/step - loss: 1.0767 - acc: 0.8298 - val_loss: 1.0693 - val_acc: 0.8325
Epoch 6/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0782 - acc: 0.8298
Epoch 00006: val_acc did not improve from 0.83250
3273/3273 [==============================] - 2527s 772ms/step - loss: 1.0781 - acc: 0.8298 - val_loss: 1.0710 - val_acc: 0.8321
Epoch 7/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0784 - acc: 0.8298
Epoch 00007: val_acc did not improve from 0.83250
3273/3273 [==============================] - 2538s 775ms/step - loss: 1.0784 - acc: 0.8298 - val_loss: 1.0741 - val_acc: 0.8321
Epoch 8/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0807 - acc: 0.8298
Epoch 00008: val_acc did not improve from 0.83250
3273/3273 [==============================] - 2540s 776ms/step - loss: 1.0807 - acc: 0.8298 - val_loss: 1.0750 - val_acc: 0.8324
Epoch 9/10
3272/3273 [============================>.] - ETA

In [50]:
model.save_weights('seq2seq_attention_weights_epoch_30.h5')
encoder_model.save_weights('seq2seq_attention_encoder_weights_epoch_30.h5')
decoder_model.save_weights('seq2seq_attention_decoder_weights_epoch_30.h5')

### After 30 epochs

In [51]:
chatbot.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi
Hi.
You: How are you?
Fine.
You: Great!
I am sorry i am not sure i am not going to be a little bit.
You: That's okay.
I am sorry i am not going to be a little bit.
You: Where are you?
I am going to be a couple of hours.
You: What's your name?
I am not sure.
You: Nice to meet you.
I am sorry i am not going to be a little bit.
You: You don't need to be sorry.
I am not sure i am not going to be a little bit.
You: My name is Joe.
I am sorry i am not sure i am not going to be a good idea.
You: Okay.
I am sorry i am not going to be a little bit.
You: I love you.
I am sorry.
You: Don't be.
I am not sure i am not going to be a little bit.
You: What color is the sky?
It is a.
You: What is your favourite color?
I am not sure.
You: Do you like blue?
I do not know i am not a little bit i am not sure i am not going to be a little bit.
You: Bye!
Hi bye.
You: 
Bye!


In [52]:
chatbot.train_model(seq_gen_train, seq_gen_val, save_name='seq2seq_attention_4', epochs=10)

Epoch 1/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0872 - acc: 0.8296
Epoch 00001: val_acc improved from -inf to 0.83154, saving model to ./checkpoints/weights-seq2seq_attention_4-best.hdf5


3273/3273 [==============================] - 2550s 779ms/step - loss: 1.0872 - acc: 0.8296 - val_loss: 1.0826 - val_acc: 0.8315
Epoch 2/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0884 - acc: 0.8296
Epoch 00002: val_acc did not improve from 0.83154
3273/3273 [==============================] - 2520s 770ms/step - loss: 1.0884 - acc: 0.8296 - val_loss: 1.0809 - val_acc: 0.8315
Epoch 3/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0903 - acc: 0.8295
Epoch 00003: val_acc did not improve from 0.83154
3273/3273 [==============================] - 2519s 770ms/step - loss: 1.0903 - acc: 0.8295 - val_loss: 1.0817 - val_acc: 0.8315
Epoch 4/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0905 - acc: 0.8295
Epoch 00004: val_acc did not improve from 0.83154
3273/3273 [==============================] - 2528s 772ms/step - loss: 1.0906 - acc: 0.8295 - val_loss: 1.0884 - val_acc: 0.8309
Epoch 5/10
3272/3273 [============================>.] - ETA

3273/3273 [==============================] - 2539s 776ms/step - loss: 1.0930 - acc: 0.8295 - val_loss: 1.0866 - val_acc: 0.8319
Epoch 7/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0940 - acc: 0.8294
Epoch 00007: val_acc did not improve from 0.83193
3273/3273 [==============================] - 2539s 776ms/step - loss: 1.0941 - acc: 0.8294 - val_loss: 1.0927 - val_acc: 0.8317
Epoch 8/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0956 - acc: 0.8294
Epoch 00008: val_acc did not improve from 0.83193
3273/3273 [==============================] - 2542s 777ms/step - loss: 1.0956 - acc: 0.8294 - val_loss: 1.0929 - val_acc: 0.8314
Epoch 9/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0965 - acc: 0.8294
Epoch 00009: val_acc did not improve from 0.83193
3273/3273 [==============================] - 2539s 776ms/step - loss: 1.0964 - acc: 0.8294 - val_loss: 1.0867 - val_acc: 0.8317
Epoch 10/10
3272/3273 [============================>.] - ET

In [53]:
model.save_weights('seq2seq_attention_weights_epoch_40.h5')
encoder_model.save_weights('seq2seq_attention_encoder_weights_epoch_40.h5')
decoder_model.save_weights('seq2seq_attention_decoder_weights_epoch_40.h5')

In [54]:
chatbot.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi
Hi.
You: How are you?
I am not going to be a good idea.
You: Great!
I am not sure.
You: That's okay.
I am not going to be a good man.
You: Where are you?
I am going to work.
You: What's your name?
I am not.
You: Nice to meet you.
I am not going to be a good time.
You: You don't need to be sorry.
I am not sure i am not going to be a good time.
You: My name is Joe.
I am not sure you are not.
You: Okay.
I am not going to be a good man.
You: I love you.
I am not a very good thing.
You: What color is the sky?
It is a good idea.
You: What is your favourite color?
I am not a very good thing.
You: Do you like blue?
Sure.
You: Bye!
Goodnight.
You: 
Bye!


In [55]:
chatbot.train_model(seq_gen_train, seq_gen_val, save_name='seq2seq_attention_5', epochs=10)

Epoch 1/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0966 - acc: 0.8293
Epoch 00001: val_acc improved from -inf to 0.83156, saving model to ./checkpoints/weights-seq2seq_attention_5-best.hdf5


3273/3273 [==============================] - 2557s 781ms/step - loss: 1.0966 - acc: 0.8293 - val_loss: 1.0929 - val_acc: 0.8316
Epoch 2/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0961 - acc: 0.8294
Epoch 00002: val_acc did not improve from 0.83156
3273/3273 [==============================] - 2540s 776ms/step - loss: 1.0961 - acc: 0.8294 - val_loss: 1.0909 - val_acc: 0.8313
Epoch 3/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0961 - acc: 0.8293
Epoch 00003: val_acc did not improve from 0.83156
3273/3273 [==============================] - 2546s 778ms/step - loss: 1.0961 - acc: 0.8293 - val_loss: 1.0899 - val_acc: 0.8314
Epoch 4/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.0972 - acc: 0.8293
Epoch 00004: val_acc did not improve from 0.83156
3273/3273 [==============================] - 2600s 795ms/step - loss: 1.0972 - acc: 0.8293 - val_loss: 1.0907 - val_acc: 0.8313
Epoch 5/10
3272/3273 [============================>.] - ETA

3273/3273 [==============================] - 2558s 782ms/step - loss: 1.1004 - acc: 0.8293 - val_loss: 1.0918 - val_acc: 0.8317
Epoch 8/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.1013 - acc: 0.8292
Epoch 00008: val_acc did not improve from 0.83172
3273/3273 [==============================] - 2552s 780ms/step - loss: 1.1013 - acc: 0.8292 - val_loss: 1.0999 - val_acc: 0.8313
Epoch 9/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.1020 - acc: 0.8293
Epoch 00009: val_acc improved from 0.83172 to 0.83200, saving model to ./checkpoints/weights-seq2seq_attention_5-best.hdf5


3273/3273 [==============================] - 2561s 782ms/step - loss: 1.1021 - acc: 0.8292 - val_loss: 1.0925 - val_acc: 0.8320
Epoch 10/10
3272/3273 [============================>.] - ETA: 0s - loss: 1.1028 - acc: 0.8292
Epoch 00010: val_acc improved from 0.83200 to 0.83220, saving model to ./checkpoints/weights-seq2seq_attention_5-best.hdf5


3273/3273 [==============================] - 2546s 778ms/step - loss: 1.1028 - acc: 0.8292 - val_loss: 1.0930 - val_acc: 0.8322


In [56]:
model.save_weights('seq2seq_attention_weights_epoch_50.h5')
encoder_model.save_weights('seq2seq_attention_encoder_weights_epoch_50.h5')
decoder_model.save_weights('seq2seq_attention_decoder_weights_epoch_50.h5')

In [57]:
chatbot.chat()

Hi, let's talk!
To end the conversation, enter an empty line. :)
You: Hi.
Hi.
You: How are you?
Fine.
You: Great!
I am not going to be a little bit of a little girl.
You: That's okay.
I am not going to be a little bit of a little girl.
You: Where are you?
I am going to have to go home.
You: What's your name?
I am not.
You: Nice to meet you.
I am not going to be.
You: You don't need to be sorry.
I am not a little of my life.
You: My name is Joe.
I am not a man.
You: What are you?
I am going to have to be a little bit of a little bit.
You: Okay.
I am not going to be.
You: I love you.
I am not a little girl.
You: What color is the sky?
It is a good idea.
You: What is your favourite color?
I am not a little girl.
You: Do you like blue?
I do not know.
You: Bye!
I am sorry.
You: Heidelberg is a beautiful city.
I am not a little girl.
You: Are you a little boy?
No.
You: Are you a little girl?
No.
You: Are you a robot?
No.
You: When did you go home?
I do not know i am not going to be a little 